In this project, we want to build a chatbot to help answer a few different questions about inventory for an online laptop store.

Here are some queries that we will want to answer:

 - Given a laptop id, find the corresponding data.
 - Given a gift card, find whether a customer can use the gift to buy up to two laptops.
 - Identify all laptops whose price falls within a given budget.
 
<b> Open dataset </b>

In [1]:
import csv
with open('laptops.csv') as file:
    file_read = csv.reader(file)
    rows = list(file_read)
    header = rows[0]
    rows = rows[1:]
    print(header)
    print(rows[0])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']


<b>Create Inventory class to process data</b>

In [2]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file = csv.reader(file)
            rows = list(read_file)
            self.header = rows[0]
            self.rows = rows[1:]
            
            #Look up by id
            self.id_to_row = {}
            for row in self.rows:         
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


<b>Compare performace of the two get_laptop_from_id methods</b>

The dictionary method should be faster with time complexity of O(1) while the loop method has time complexity of O(R).

In [3]:
import time                                                        
import random                                                       

ids = [str(random.randint(10000, 99999)) for _ in range(10000)] 

inventory = Inventory('laptops.csv')                               

total_time_loop = 0                                              
for identifier in ids:                                              
    start = time.time()                                            
    inventory.get_laptop_from_id(identifier)                       
    end = time.time()                                            
    total_time_loop += end - start                             
    
total_time_dict = 0                                              
for identifier in ids:                                             
    start = time.time()                                          
    inventory.get_laptop_from_id_fast(identifier)                
    end = time.time()                                               
    total_time_dict += end - start                                 
    
print(total_time_loop)                                          
print(total_time_dict)

0.9924063682556152
0.0046541690826416016


<b>Gift Card Amount Check & Find Laptop within budget</b>

1. Gift Card check:
The store offers gift cards that customer can use to buy up to two laptops with a single time usage. You don't want to make a customer feel cheated, so there must be at least one way to spend full amount on a gift card, whether by buying one or two laptops.


2. Find a laptop within D dollars:
Filter out all laptops out of budget.

In [4]:
class Inventory():
    
    def __init__(self, csv_filename):
        
        with open(csv_filename) as file:
            read_file = csv.reader(file)
            rows = list(read_file)
            self.header = rows[0]
            self.rows = rows[1:]
            
            #Look up by id
            self.id_to_row = {}
            for row in self.rows:         
                self.id_to_row[row[0]] = row
            
            #Gift card amount check
            self.prices = set() #not dictionary
            for row in self.rows:         
                self.prices.add(int(row[-1]))
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def gift_card_sufficient(self, dollars):
        for price in self.prices:                    
            if dollars - price in self.prices:
                return "You can buy 2 laptops."
        if dollars in self.prices:                   
            return "You can only buy 1 laptop."
        return "Gift card is not sufficient."       
    
    def laptop_in_budget(self, budget):
        in_budget = [row for row in self.rows if int(row[-1]) <= budget]
        if in_budget:
            message = f"There are {len(in_budget)} laptops in budget:\n"
            for row in in_budget:
                message += f"{row}\n"
            return message
        else: return None
    
inventory = Inventory('laptops.csv')
print(inventory.gift_card_sufficient(1000))
print(inventory.laptop_in_budget(200))

You can buy 2 laptops.
There are 5 laptops in budget:
['7667029', 'Asus', 'Vivobook E200HA', 'Netbook', '11.6', '1366x768', 'Intel Atom x5-Z8350 1.44GHz', '2GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '0.98kg', '191']
['4366200', 'Asus', 'E402WA-GA010T (E2-6110/2GB/32GB/W10)', 'Notebook', '14', '1366x768', 'AMD E-Series E2-6110 1.5GHz', '2GB', '32GB Flash Storage', 'AMD Radeon R2', 'Windows 10', '1.65kg', '199']
['3840240', 'Acer', 'Chromebook C910-C2ST', 'Notebook', '15.6', '1366x768', 'Intel Celeron Dual Core 3205U 1.5GHz', '2GB', '16GB SSD', 'Intel HD Graphics', 'Chrome OS', '2.19kg', '199']
['1478754', 'Vero', 'V131 (X5-Z8350/4GB/32GB/FHD/W10)', 'Notebook', '13.3', 'Full HD 1920x1080', 'Intel Atom X5-Z8350 1.44GHz', '4GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '1.35kg', '196']
['3564228', 'Acer', 'C740-C9QX (3205U/2GB/32GB/Chrome', 'Netbook', '11.6', '1366x768', 'Intel Celeron Dual Core 3205U 1.5GHz', '2GB', '32GB SSD', 'Intel HD Graphic

<b>Build chatbot</b>


Now we can build a chatbot to help customer answer those questions above.

In [ ]:
def chatbot(inventory):
    print("Welcome to the Laptop Store chatbot!")
    print("You can ask:")
    print("- Find laptop by id <laptop_id>")
    print("- How many laptops can I buy with <amount>")
    print("- Show laptops under <amount>")
   
    while True:
        query = input("You: ").strip().lower()

        if query.startswith("find laptop"):
            # Extract laptop id from the prompt
            laptop_id = query.split()[-1] 
            laptop = inventory.get_laptop_from_id_fast(laptop_id)
            if laptop:
                formatted = "\n".join(f"{h}: {v}" for h, v in zip(inventory.header, laptop))
                print("Bot:\n" + formatted)
            else:
                print("Bot: Laptop ID not found.")

        elif query.startswith("how many laptops"):
            try:
                amount = int(query.split()[-1])
                result = inventory.gift_card_sufficient(amount)
                print("Bot:", result)
            except ValueError:
                print("Bot: Please provide a valid gift card amount.")

        elif query.startswith("show laptops under"):
            try:
                budget = int(query.split()[-1])
                result = inventory.laptop_in_budget(budget)
                if result:
                    print("Bot:", result)
                else:
                    print("Bot: No laptops found within that budget.")
            except ValueError:
                print("Bot: Please provide a valid budget amount.")

        else:
            print("Bot: Sorry, I didn't understand that. Please try again.")
            
chatbot(inventory)

Welcome to the Laptop Store chatbot!
You can ask:
- Find laptop by id <laptop_id>
- How many laptops can I buy with <amount>
- Show laptops under <amount>
You: find laptop 3564228
Bot:
Id: 3564228
Company: Acer
Product: C740-C9QX (3205U/2GB/32GB/Chrome
TypeName: Netbook
Inches: 11.6
ScreenResolution: 1366x768
Cpu: Intel Celeron Dual Core 3205U 1.5GHz
Ram: 2GB
Memory: 32GB SSD
Gpu: Intel HD Graphics
OpSys: Chrome OS
Weight: 1.3kg
Price: 174
